# Imputación Ingenua de Missing Value
##### Es un método que permite imputar valores perdidos de una forma ingenua pero eficiente. El requisito de esta técnica es tener una variable target dicotomica o categorica, en la cual, posteriormente a la imputación se realice un modelo con dicha variable. 
##### Mayor información: https://github.com/fmansillaib/python_MsNAIVE
##### Personal Page: https://francomansilla.com

#### Cargar la Base de Datos 

In [1]:
import pandas as pd
df = pd.read_excel("/Volumes/GoogleDrive-111868847232940162537/Mi unidad/INVESTIGACION-TRABAJOS/Codigos Stata/MsNAIVE/BD MsNAIVE.xlsx")
df.head()

,fraude,ing_x1,ing_x2,ing_x3,cost_x1,cost_x2,cost_x3,cost_flag,muestra
0,0,90.0,1.0,245.70,NaN,0.0,4.30920,1,Train
1,1,56.0,3.0,NaN,35.28,0.0,2.68128,1,Train
2,0,NaN,2.0,92.82,21.42,NaN,1.62792,1,Train
3,1,50.0,NaN,136.50,31.50,NaN,2.39400,1,Train
4,1,91.0,NaN,248.43,57.33,1.0,4.35708,1,Train


##### ---- ~ -----
##### Nota 1: La variable fraude es la variable target, para una posterior modelelo supervisado.
##### Nota 2: Desde la variable gast_x# hasta cost_x# son variables que tiene missing value (NaN) lo cual habra que imputar.
##### ---- ~ -----

#### Paso 2: Análisis de Missing Value

In [2]:
for i in df.columns.tolist():
    porc_i = round(df[i].isnull().sum()/len(df[i]),4)
    print(i+" =", porc_i)

fraude = 0.0
ing_x1 = 0.0773
ing_x2 = 0.0676
ing_x3 = 0.0773
cost_x1 = 0.1111
cost_x2 = 0.0966
cost_x3 = 0.087
cost_flag = 0.0
muestra = 0.0


##### ---- ~ -----
##### Nota 3: Si encontramos una variable sobre un estandar, por ejemplo, 15% de missing value, se puede evaluar eliminar la variable.
##### ---- ~ -----

##### Paso 3: Imputación de Missing Value

##### Paso 3.1. Criterios a considerar:
###### 1. Considera variables con 2 o más valores unicos.
###### 2. Considera variables con menos del umbral de missing value; este caso 15%.
###### 3. Considera variables dicotomicas, es decir, contenga 2 valores discretos (0 ó 1) u otra combinación.
###### 4. Considera variables categoricas, es decir, variable contenga 3 o más valores discretos. 
###### 5. Considera variables categoricas con caracter continuo, es decir, variable contenga # o más valores discretos. 

##### ---- ~ -----

##### Paso 3.2. Descripción Imputación
###### Si se tiene dos clases de target [0;1], se calculara la mediana de la variable a imputar cuando clase del target es 0; la mismo para la clase target es 1. El calculo se considera la muestra TRAIN para aplicar toda la muestra.

##### ---- ~ -----

##### Paso 3.3. Código de Imputación

In [3]:
# df a imputar
df_new = df.copy()

# df con valores sin na y filtro
df_var = df.filter(regex = 'ing|cost' , axis = 1)
# o
#Señalar todas las variables que no necesitar ser imputadas.
var_nd = ['ventas', 'muestra']
#Variables a imputar
var_x = [x for x in df_new.columns if x not in var_nd]

In [4]:
for i in df_var.columns.tolist(): #ó poner for i in var_x:
    
    umbral_unique = 10 # Scalar
    umbral_nan = 0.1 # Scalar
    target = 'fraude' # Columna df.
    muestra = 'muestra' # Columna df.
    ventana = 'Train' # Atributo de la Columna Muestra.
    
    #Criterio 1: Umbral de Missing Value
    porc_i = round(df[i].isnull().sum()/len(df[i]),4)
    
    #Criterio 2: Cantidad de Valores Unicos
    unique_val = df[i].loc[df[muestra]==ventana].nunique()
    
    # Imputación por Regresión variables continuas y categorias de caracter continuas.
    if porc_i <= umbral_nan and unique_val >= umbral_unique:
        
        print('La variable '+i ,'fue imputada con MEDIANA DE CLASE', 'con',+unique_val,'valores únicos y',porc_i, 'valores perdidos.' )        
        
        for j in df[target].unique():
            
            # Calculo de mediana de la variable(i) en función de la clase target(j).
            median_xclass = df[i].loc[(df[target]== j) & (df[muestra]==ventana)].median()
            # Reemplaza los NaN a las variable(i) en función de la clase target(j).
            df_new[i].loc[(df[i].isna()) & (df[target]== j)] = median_xclass
                        
    # Elimina variables con MISSING VALUE mayor al Umbral y un único valor en los valores únicos.                                            
    elif porc_i > umbral_nan or unique_val <= 1:
        
        print('La variable '+i ,'fue ELIMINADA por tener',porc_i, 'valores perdidos.' )  
        
        df_new.drop(i, axis = 1, inplace = True)
        
        
    # Imputación por CATEGORIA FALTANTE a varaibles dicotomicas y variables con menos de 15 categorias.
    elif unique_val > 1  & unique_val < umbral_unique:
        
        print('La variable '+i ,'fue imputada por VALOR FALTANTE', 'con',+unique_val,'valores únicos y',porc_i, 'valores perdidos.' )        
        
        #Calculo máximo valor
        value_max = df[i].max()
        #Valor máximo +1
        value_mas1 = value_max + 1
        #Reemplazar NaN por el Valor asignado.
        df_new[i].fillna(value_mas1, inplace = True)
#FIN

La variable ing_x1 fue imputada con MEDIANA DE CLASE con 64 valores únicos y 0.0773 valores perdidos.
La variable ing_x2 fue imputada por VALOR FALTANTE con 3 valores únicos y 0.0676 valores perdidos.
La variable ing_x3 fue imputada con MEDIANA DE CLASE con 67 valores únicos y 0.0773 valores perdidos.
La variable cost_x1 fue ELIMINADA por tener 0.1111 valores perdidos.
La variable cost_x2 fue imputada por VALOR FALTANTE con 2 valores únicos y 0.0966 valores perdidos.
La variable cost_x3 fue imputada con MEDIANA DE CLASE con 66 valores únicos y 0.087 valores perdidos.
La variable cost_flag fue ELIMINADA por tener 0.0 valores perdidos.


/Users/francomansilla/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [5]:
df_new.isnull().sum()

fraude     0
ing_x1     0
ing_x2     0
ing_x3     0
cost_x2    0
cost_x3    0
muestra    0
dtype: int64

In [6]:
df_new.head()

,fraude,ing_x1,ing_x2,ing_x3,cost_x2,cost_x3,muestra
0,0,90.0,1.0,245.70,0.0,4.30920,Train
1,1,56.0,3.0,158.34,0.0,2.68128,Train
2,0,49.0,2.0,92.82,2.0,1.62792,Train
3,1,50.0,4.0,136.50,2.0,2.39400,Train
4,1,91.0,4.0,248.43,1.0,4.35708,Train
